In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import cv2
import torch
import torchvision

import pandas as pd
import numpy as np
import seaborn as sns 
import albumentations as A
import matplotlib.pyplot as plt

from pathlib import Path
from tqdm.cli import tqdm
from pytorch_toolbelt.inference import tta

from src.data import EyeDataset
from src.utils import load_splits
from src.utils import custom_to_std_tensor
from src.utils import denormalize
from src.utils import eye_blend
from src.config import load_config_from_ckpt
from src.models import ModelEnsemble
from src.train import validation

In [3]:
data_folder = Path("data")
folds_folder = Path("folds")
models_folder = Path("models")
configs_folder = Path("configs")
figs_folder = Path("figs")
output_folder = Path("output")
submissions_folder = Path("submissions")

test_df = pd.read_csv(data_folder / "test.csv")

## Single Model validation

In [4]:
config_filename = configs_folder / "baseline.yml"
checkpoint_filename = models_folder / "baseline-epoch-38-ckpt.pt"
config = load_config_from_ckpt(config_filename, checkpoint_filename)

model = config.model
model = tta.TTAWrapper(config.model, tta.fliplr_image2mask) 


config.dataloaders.val.dataset.return_pos = False
dataloader = config.dataloaders.val

Overwriting device = gpu (was gpu)
Overwriting non-existing attribute checkpoint.filename = models/baseline-epoch-38-ckpt.pt
Overwriting non-existing attribute checkpoint.filename = models/baseline-epoch-38-ckpt.pt
Overwriting non-existing attribute checkpoint.model = True
{'model': {'name': 'models.smp.Unet', 'params': {'encoder_name': 'efficientnet-b2', 'encoder_weights': 'imagenet', 'in_channels': 1, 'classes': 4, 'activation': None}}, 'device': 'gpu', 'criterion': {'name': 'L.JaccardLoss', 'params': {'mode': 'multiclass'}}, 'optimizer': {'name': 'torch.optim.Adam', 'params': {'lr': 0.001}}, 'scheduler': {'name': 'torch.optim.lr_scheduler.ReduceLROnPlateau', 'params': {'patience': 10, 'factor': 0.5, 'mode': 'max'}}, 'dataloaders': {'train': {'name': 'torch.utils.data.DataLoader', 'params': {'dataset': {'name': 'data.EyeDataset', 'params': {'df': {'name': 'utils.load_splits', 'params': {'folds_folder': {'path': 'folds'}, 'val_folds': [0], 'only_train': True}}, 'mode': 'train', 'trans

validation phase:   0%|          | 0/101 [00:00<?, ?it/s]

Load model from state dict


validation phase: 100%|██████████| 101/101 [00:11<00:00,  8.96it/s]


{'val_loss': 0.05054121527505751,
 'mean_iou': 0.9136595248582944,
 'mean_with_bg_iou': 0.9335828500600597,
 'sclera_iou': 0.8811745228458993,
 'iris_iou': 0.950562122449353,
 'pupil_iou': 0.9092418634476354}

In [ ]:
def validation(model, device, criterion, metrics, dataloader, pbar_desc="validation phase"):
    model.eval()
    val_loss = 0.0
    val_metrics = {k: 0.0 for k, v in metrics.items()}
    
    for origins, masks, pos in tqdm(dataloader, desc=pbar_desc):
        num = origins.size(0)

        origins = origins.to(device)
        masks = masks.to(device)

        with torch.no_grad():
            outs = model(origins)
        
            val_loss += criterion(outs, masks).item() * num
            val_metrics = {k: v + metrics[k](masks, outs).item() * num for k, v in val_metrics.items()}
        

    val_loss = val_loss / len(dataloader.sampler)
    val_metrics = {k: v / len(dataloader.sampler) for k, v in val_metrics.items()}
    
    return {
        "val_loss": val_loss,
        **val_metrics,
    }

In [ ]:
validation(
    model,
    config.device,
    config.criterion,
    config.metrics,
    config.dataloaders.val,
)